### Алгоритмы поиска соседей (в глубину и в ширину)

Хорошее описание (с примерами на Java) находится [здесь](https://habr.com/ru/post/504374/). Хорошее описание поиска в глубину с примерами на Питоне (с применением классов) начинается [здесь](https://aliev.me/runestone/Graphs/GeneralDepthFirstSearch.html), поиска в ширину - [здесь](https://aliev.me/runestone/Graphs/ImplementingBreadthFirstSearch.html).

*Постановка задачи*: Необходимо найти путь, соединяющий две вершины.

*Альтернативная постановка задачи*: Необходимо найти путь минимальной длины, соединяющий две вершины.

Считаем, что начальная вершина не совпадает с конечной.

*Решение обходом в глубину*: 
1. Назначаем текущей начальную вершину, создаем пустой список с путем, помечаем все вершины графа как непройденные.
2. Если один из соседей текущей вершины является конечной вершиной - добавляем конечную вершину и себя в список с путем и возвращаем успех. В противном случае выполняем п. 3.
3. Помечаем себя как пройденную вершину. Перебираем всех соседей, отмеченных как непройденные, назначая их текущей вершиной и предлагая им найти путь до конечной вершины. Если кто-то из соседей возвращает успех, добавляем себя в список с путем и возвращаем успех. Если все соседи вернули неуспех, возвращаем неуспех.

*Решение обходом в ширину* (реализация в один список):
1. Создаем список текущих вершин, помещаем в него начальную вершину. Повторяем Шаг 2 пока список текущих вершин не пройден до конца.
2. Для текущей вершины добавляем ее соседей, которых еще нет в списке пройденных вершин. Если кто-то из соседей является конечной вершиной, возвращаем успех.
3. Если все вершины пройдены - возвращаем неуспех.

Последняя реализация алгоритма не возвращает путь, но может использоваться в качестве алгоритма нахождения компоненты связности для ненаправленного графа - в списке текущих вершин окажутся все вершины, достижимые из данной.

Заметьте, можно искать путь, а можно просто находить его длину. Первый алгоритм подходит и для первой, и для второй задачи, а вот второй алгоритм подходит только для решения второй.

Давайте рассмотрим реализацию алгоримтов на практике.

In [1]:
# Будем хранить граф как список вершин и список дуг для данной вершины. 
# Считаем, что граф ненаправленный.
V = ['V0', 'V1', 'V2', 'V3', 'V4', 'V5']
E = [[1, 2], [0, 3, 5], [0, 3], [1, 2, 4, 5], [3], [3, 1]]
visited = [False] * len(V)

In [2]:
# Функция поиска первого попавшегося пути до вершины where.
# Текущая вершина с номером node.
def findPath(node, where):
    # Пусть каждая вершина проверяет себя сама.
    if where == node:
        return True, [V[node]]
    # Текущая вершина уже посещена.
    visited[node] = True
    # Переберем всех непосещенных потомков.
    for n in E[node]:
        if visited[n]:
            continue
        ret, l = findPath(n, where)
        # Путь найден!
        if ret:
            l1 = [V[node]]
            l1.extend(l)
            return True, l1
    return False, None

In [3]:
# Ищем путь из вершины 0 в вершину 5.
visited = [False] * len(V)
findPath(0, 5)

(True, ['V0', 'V1', 'V3', 'V5'])

Как видно, алгоритм нашел путь, но первыый, который ему попался. Можно было просто вернуть `True`, покаывая, что пусть есть, а не стараться и не возвращать сам путь.

Теперь модифицируем фуункцию так, чтобы она искала более короткий путь. Для этого придется опросить всех потомков и выбрать из нескольких предложенных путей самый короткий.

Кстати, использовать глобальные переменные - не самая лучшая практика. Исправим и этот недочет.

In [4]:
# Функция поиска самого короткого пути до вершины where.
# Текущая вершина с номером node.
def findPath2(verticies, edges, visits, cur_node, where):
    # Пусть каждая вершина проверяет себя сама.
    if where == cur_node:
        return [verticies[cur_node]]
    # Текущая вершина уже посещена.
    visits[cur_node] = True
    # Пока ничего не найдено, длина самого короткого пути назначается длиннее списка вершин.
    # Зачем возвращать булевское значение, если список сам всё покажет?
    min_len = len(verticies) + 10
    l1 = None
    # Переберем всех непосещенных потомков.
    for n in edges[cur_node]:
        if visits[n]:
            continue
        l = findPath2(verticies, edges, visits, n, where)
        if l != None and len(l) < min_len:
            l1 = [verticies[cur_node]]
            l1.extend(l)
    return l1

In [5]:
visited = [False] * len(V)
findPath2(V, E, visited, 0, 5)

['V0', 'V1', 'V5']

Попадание! Теперь мы возвращаем самый короткий маршрут.

Убедимся, что алгоритм не зацикливается при прохождении цикла в графе.

In [6]:
# Будем хранить граф как список вершин и список дуг для данной вершины. 
# Считаем, что граф ненаправленный.
V2 = ['0', '1', '2', '3', '4', '5', '6']
E2 = [[6, 1, 2], [0, 3, 5], [0, 3], [1, 2, 4, 5], [3], [3, 1], [0]]
visited = [False] * len(V2)
findPath2(V2, E2, visited, 0, 5)

['0', '1', '5']

Теперь посмотрим на работу алгоритма поиска в ширину.

In [7]:
# Функция поиска самого короткого пути до вершины where.
# Текущая вершина с номером node.
def findPath3(edges, start_node, where):
    visits = [start_node]
    i = 0
    while i < len(visits):
        if visits[i] == where:
            return True
        for n in edges[visits[i]]:
            if n not in visits:
                visits.append(n)
        i += 1
    return False
    

In [8]:
print(findPath3(E, 0, 5))
print(findPath3(E2, 0, 5))

True
True


Обратите внимание, что алгоритм может возвращать путь от одной вершины до другой, если мы его модифицируем. С одной стороны, он всегда находит самый короткий путь - как только "волна", которая началась из начальной вершины, "коснется" конечной вершины, путь будет найден. Так как мы всегда добавляем вершины следующего слоя, то от них найти более короткий путь не получится. Кстати, из-за подобной аналогии алгоритм называется **волновым алгоритмом поиска пути в графе**.

С другой стороны, мы можем использовать этот факт для того, чтобы найти длину пройденного маршрута. Длина маршрута до начальной вершины равна 0. Каждая вершина знает свою длину маршрута от стартовой и может при добавлении новой вершины увеличить длину маршрута до нее на 1.

In [9]:
# Функция поиска самого короткого пути до вершины where.
# Текущая вершина с номером node.
def findPath4(edges, start_node, where):
    visits = [(start_node, 0)]
    i = 0
    while i < len(visits):
        if visits[i][0] == where:
            return visits[i][1]
        for n in edges[visits[i][0]]:
            if n not in visits:
                visits.append((n, visits[i][1] + 1))
        i += 1
    return -1
    
   

In [10]:
print(findPath4(E, 0, 5))
print(findPath4(E2, 0, 5))

2
2


Однако "фронт волны" в данном лгоритме растет как квадрат от расстояния от начальной вершины. Если граф большой, а конечная вершина находится далеко от начальной, то поиск может занимать много времени. Поэтому вместо того, чтобы пускать одну волну, мы запустим две: одну от начальной вершины, вторую от конечной. Как известно, сумма квадратов меньше квадрата суммы, поэтому количество проделанной работы будет значительно меньше.

Этот алгоритм называется **алгоритмом встречного распространения волны** или [**волновым алгоритмом Ли**](https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC_%D0%9B%D0%B8). Результаты работы алгоритма можно продемонстрировать при помощи рисунков.

![](https://upload.wikimedia.org/wikipedia/commons/9/93/Lee_wave_4.png)

![](https://upload.wikimedia.org/wikipedia/commons/4/4e/Lee_wave_8.png)

В описании выше мы подразумевали, что все расстояния между соседними вершинами равны 1. Но на практике граф может быть взвешенным, то есть стоимость перехода может отличаться. Тогда можно рассчитать не длину, а стоимость маршрута (часто также называемую длиной). Если все стоимости переходов положительные, мы можем использовать

### [Алгоритм Дейкстры](https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC_%D0%94%D0%B5%D0%B9%D0%BA%D1%81%D1%82%D1%80%D1%8B)

В случае взвешенного графа, самое маленькое количество переходов не гарантирует нам самой низкой стоимости пути. Используя аналогии - очень длинная тропинка вокруг болота проще, чем дорога через него. 

Суть алгоритма Дейкстры заключается в следующем. Нам не нужно перебирать все вершины подряд. Лучше выбрать из всех такую, маршрут до которой на данный момент минимален. Есть шанс, что ведущий из нее путь окажется самым коротким. Если стоимость переходов из выбранной вершины больше, чем у других вершин, на следующем шаге мы выберем другого кандидата. 

Вторым предположениемм алгоритма является то, что окружная дорога до какой-то врешины может быть дешевле, чем прямая. Это означает, что мы можем найти более дешевый маршрут за большее число ходов. Таким образом, мы должны быть готовы к тому, что стоимости маршрутов до вершин могут уменьшаться.

Сам алгоритм выглядит следующим образом.

**Инициализация**
- Метка начальной вершины равна 0, метки остальных вершин — бесконечности. Это отражает тот факт, что расстояния до других вершин пока неизвестны.
- Все вершины графа помечаются как непосещённые.

**Шаг алгоритма**
- Если все вершины посещены, алгоритм завершается.
- В противном случае, из ещё не посещённых вершин выбирается вершина _u_, имеющая минимальную метку.
- Мы рассматриваем всевозможные маршруты, в которых _u_ является предпоследним пунктом. Для каждого соседа вершины _u_, кроме отмеченных как посещённые, рассмотрим новую длину пути, равную сумме значений текущей метки _u_ и длины ребра, соединяющего _u_ с этим соседом.
- Если полученное значение длины меньше значения метки соседа, заменим значение метки полученным значением длины. Рассмотрев всех соседей, пометим вершину _u_ как посещённую и повторим шаг алгоритма. 

В качестве промежуточной точки рассмотрим
### [Алгоритм Флойда-Уоршела](https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC_%D0%A4%D0%BB%D0%BE%D0%B9%D0%B4%D0%B0_%E2%80%94_%D0%A3%D0%BE%D1%80%D1%88%D0%B5%D0%BB%D0%BB%D0%B0)

Данный алгоритм предназначен для поиска длины всех путей в графе.

Пусть граф задан матрицей связности, причем в ячейке матрицы $M_{i,j}$ расположена стоимость перехода между из вершины $i$ в вершину $j$, равную бесконечности в случае, если вершины не связаны. Идея алгоритма заключаетсяя в следующем. Путь от одной вершины к другой через третью вершину может оказаться дешевле, чем переход напрямую. Тогда мы можем заменить известную стоимость перехода на новую, меньшую. Проделав такой шаг для всей матрицы мы сможем уменьшить стоимость преехода для части дуг. Но после этого может получиться, что переход из первой вершины в третью или из третьей во вторую будет дешевле через какую-то четвертую. То есть если повторить алгоритм еще раз, можно снова сократить стоимость перехода. В итоге мы приходим к алгоритму со следующим кодом.

`for k in range(n):
  for i in range(n):
    for j in range(n)
      W[i][j] = min(W[i][j], W[i][k] + W[k][j])`
      
      
Аналогичную идею использует 

### [Алгоритм Беллмана-Форда](https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC_%D0%91%D0%B5%D0%BB%D0%BB%D0%BC%D0%B0%D0%BD%D0%B0_%E2%80%94_%D0%A4%D0%BE%D1%80%D0%B4%D0%B0)

который ищет путь между двумя вершинами в графе с дугами отрицательным весом. Его суть заключается в том, что мы сокращаем известную стоимость перехода из начальной вершины в конечную (а по дороге и в остальные вершины), проходя через промежуточные вершины. Так как число шагов фиксируется, циклы с отрицательным весом прохождения, не дают нам свалиться в минус бесконечность.



Теперь разберем еще несколько алгоритмов на графах.

### Алгоритм поиска подграфа в графе

Порождённый подграф графа — это другой граф, образованный из подмножества вершин графа вместе со всеми рёбрами, соединяющими пары вершин из этого подмножества. 

Если использовать прямые методы поиска, то сложность задачи растет экспоненциально (хотя скорее по факториалу, но при большом размере графа и относительно небольшом размере подграфа можно считать, что задача экспоненциальна). Нам необходимо перебратьв все вершины исходного графа в поисках первой вершины подграфа. Далее мы перебираем все вершины кроме одной для поиска второй вершины. И так далее.

Например, нам необходимо по фотографии звездного неба определить какие именно звезды были сняты. При этом фотография будет не лучшего качества, чем имеющийся у нас атлас. Данная задача может быть сведена к задаче поиска подграфа в графе, если для каждой звезды на фото мы отметим связи с $k$ ее ближайшими соседями. В итоге получится граф, в котором у каждой вершины которого будет не более, чем $k$ вершин. В атласе мы можем выбрать значительно большее число соседей, расстояние до которых не больше определенного порога. Варьируя пороговое расстояние мы масштабируем расстояния на карте, параллельно устраняя часть звезд в зависимости от их яркости. Поиск в подобных графах займет значительное время.

Теперь используем некоторые знаний, которые помогут нам ускорить процесс. В случае со звездами это будут спектральные характеристики, относительные расстояния между ними, а также углы между дугами. Пусть на фотографии есть две звезды, расположенные рядом с третьей, и четвертая звезда, расположенная подальше. Тогда сперва найдем в атласе все звезды, с тем же спектральным классом, что и третья. Число таких звезд будет всё еще значительным, но уже гораздо меньшим. Далее найдем звезду, у которой будет три соседа с таким же соотношением расстояний и тем же спектральным классом. Таких звезд будет еще меньше. Те же рассуждения можно применить и к найденным кандидатам и их соседям. Таким образом, отфильтровывая результаты, мы сможем за меньшее число операций найти кандидатов на фото. Алгоритм придется повторить неколько раз для разных характеристик фотоаппаратуры, но если она известна заранее, можно предварительно отсеять слишком тусклые звезды, находящиеся за гранью чувствительности аппаратуры.

Таким образом, зная как устроен граф, мы можем перейти от задачи экспоненциальной сложности к более простому решению. Например, в графах с зафиксированной структурой при поиске подграфа не стоит сопоставлять вершины подграфа с вершинами графа с меньшим количеством дуг.

Еще одним примером переноса алгоритмов на графах является [**алгоритм Хиршберга**](https://habr.com/ru/post/117063/) реализующий поиск [**расстояния Левенштейна**](https://dic.academic.ru/dic.nsf/ruwiki/43819) для сравнения строк.